In [1]:
import os

# 1. Extract Keypoints
load video data and extract keypoints using mediapose into /data/pose_data.csv

In [2]:
from src import extract_keypoints as extract

# Process all MP4s from a folder and extract labeled keypoints
video_dir = os.path.join("data", "raw")
df = extract.process_directory(video_dir)

# Save result for training
csv_path = os.path.join("data", "pose_data.csv")
extract.save_dataframe(df, csv_path)


Processing left_0.mp4...
Processing left_1.mp4...
Processing left_2.mp4...
Processing neutral_0.mp4...
Processing neutral_1.mp4...
Processing neutral_2.mp4...
Processing right_0.mp4...
Processing right_1.mp4...
Processing right_2.mp4...
✅ Saved keypoints to data\pose_data.csv


# 2. Train Model
create classifier for the keypoints and save the model in models/tf_model

In [3]:
from src import train_pose_model as train
from sklearn.model_selection import train_test_split

model_dir = os.path.join("models", "tf_model")
label_map_path = os.path.join("models", "label_map.json")

# Load data
X, y = train.load_pose_data(csv_path)
y_encoded, label_encoder = train.encode_labels(y)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Build and train model
model = train.build_model(input_dim=X.shape[1], num_classes=len(label_encoder.classes_))
model.summary()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=32)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

# Save model and labels
train.save_model(model, model_dir)
train.save_label_map(label_encoder, label_map_path)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        17,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,475 (99.51 KB)

 Trainable params: 25,475 (99.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4186 - loss: 1.0801 - val_accuracy: 0.8432 - val_loss: 0.8284
Epoch 2/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7029 - loss: 0.8303 - val_accuracy: 0.8077 - val_loss: 0.5871
Epoch 3/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7974 - loss: 0.6055 - val_accuracy: 0.9497 - val_loss: 0.3303
Epoch 4/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8941 - loss: 0.3815 - val_accuracy: 0.9675 - val_loss: 0.2116
Epoch 5/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9305 - loss: 0.2700 - val_accuracy: 0.9793 - val_loss: 0.1440
Epoch 6/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9436 - loss: 0.2056 - val_accuracy: 0.9852 - val_loss: 0.1024
Epoch 7/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9399 - loss: 0.1809 - val_accuracy: 0.9763 - val_loss: 0.1209
Epoch 8/15
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9471 - loss: 0.1621 - val_accuracy: 0.9941 - val_loss:

INFO:tensorflow:Assets written to: models\tf_model\assets
